In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 데이터셋의 컬럼 숨김없이 보여주는 설정
pd.options.display.max_columns=99

In [ ]:
# 중고차 거래시 사기당했는지 아닌지 구분(분류)하는 데이터
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
train

In [ ]:
test = pd.read_csv('/kaggle/input/DontGetKicked/test.csv')
test

In [ ]:
# 전처리가 많이 들어가는경우(컬럼이 많은 경우에는) train/test를 묶어 all_data로 해주는게 편하다! 
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data['PurchDate'] = pd.to_datetime(all_data['PurchDate'])
all_data

In [ ]:
all_data['year'] = all_data['PurchDate'].dt.year
all_data['month'] = all_data['PurchDate'].dt.month
all_data['day'] = all_data['PurchDate'].dt.day

In [ ]:
# 컬럼지우기(베이스라인 잡기)
all_data_2 = all_data.drop(columns = ['IsBadBuy','PurchDate','RefId'])
all_data_2

In [ ]:
# 문자형 컬럼 숫자로 바꿔주기
# 1) Auction
all_data_2['Auction'].unique()

In [ ]:
# 직접 바꿔주기
# all_data_2['Auction'] = all_data_2['Auction'].replace({'ADESA':1,'OTHER':2,"MANHEIM":3})
# all_data_2
# --> 일일이 수동으로 바꾸다가 점수하락이 올 수 있으므로 전부 le에 맡기자

In [ ]:
# 2) Make
all_data_2['Make'].unique()

In [ ]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_data_2['Make'] = le.fit_transform(all_data_2['Make'])
all_data_2

In [ ]:
# 어떤 카테고리가 어떤 숫자가 되는지 알아보기
le.classes_
# 맨 앞쪽부터 0

In [ ]:
# 나머지 컬럼들 한번에 처리 --> for문사용
# 먼저, 문자형 컬럼에 어떤것들이 있는가?
c = all_data_2.columns[all_data_2.dtypes==object]
c

In [ ]:
for i in c :
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))
all_data_2
# 리스트 안씌웠을때:
# TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']
# 결측치(float)이 생겨 오류가 발생!
# 위의 값을 list화 시켜 NAN도 카테고리 처리한다! --> [NAN]으로

In [ ]:
all_data_2 = all_data_2.fillna(-1)

In [ ]:
# 전처리 끝났으므로 train_2,test_2 로 나누기!
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
'''
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1)
rf.fit(train_2,train['IsBadBuy'])
'''
from catboost import CatBoostClassifier
cb = CatBoostClassifier(verbose = 100)
cb.fit(train_2,train['IsBadBuy'])

In [ ]:
# result = rf.predict(test_2)
# result
result = cb.predict_proba(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')
sub

In [ ]:
# RefId를 제외하고 IsBadBuy에만 값 넣기
sub['IsBadBuy'] = result[:,1]  # 로우는 전체, 컬럼은 1(사기일 확률)만!
sub

In [ ]:
sub.to_csv('result.csv',index=False)

# 이렇게 저장해서 모델 점수를 체크했더니 점수가 낮게 나왔다!
# --> 분류문제를 0,1이 아니라 몇 %의 확률로 사기인지 아닌지 보여줘서 제출해야함!

In [ ]:
# 확률이 몇 %인지 보여주는 함수 predict_proba --> 왼쪽이 0번, 오른쪽이 1번
result = rf.predict_proba(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')
sub

In [ ]:
# result[:,1] --> 로우는 처음부터 끝까지, 컬럼은 1(사기일 확률)만
sub['IsBadBuy'] = result[:,1]
sub

In [ ]:
sub.to_csv('result.csv',index=False)